In [1]:
import pandas as pd
import numpy as np
import os

# Load the preprocessed EEG dataset
input_file = '/Users/puchku-home/KIIT SEM/PROJECT/EEG/EEG Assets/chbmit_preprocessed_data.csv' # Change this path
output_dir = '/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS' # Change this path

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Read the dataset
data = pd.read_csv(input_file)
eeg_columns = [col for col in data.columns if col != 'Outcome']

# Convert to NumPy array for faster computation
eeg_data = data[eeg_columns].values
outcomes = data['Outcome'].values

# Define window parameters
fs = 256  # Sampling frequency (Hz)
window_size = fs * 1  # 1-second window
step_size = window_size // 2  # 50% overlap

### Feature Calculation Functions
def calculate_mean(window, eeg_columns):
    return {f'{col}_mean': np.mean(window, axis=0) for col in eeg_columns}

def calculate_variance(window, eeg_columns):
    return {f'{col}_var': np.var(window, axis=0) for col in eeg_columns}

def calculate_std(window, eeg_columns):
    return {f'{col}_std': np.std(window, axis=0) for col in eeg_columns}

def calculate_min_max(window, eeg_columns):
    return {f'{col}_min': np.min(window, axis=0) for col in eeg_columns} | \
           {f'{col}_max': np.max(window, axis=0) for col in eeg_columns}

def calculate_ptp(window, eeg_columns):
    return {f'{col}_ptp': np.ptp(window, axis=0) for col in eeg_columns}

def calculate_energy(window, eeg_columns):
    return {f'{col}_energy': np.sum(window**2, axis=0) for col in eeg_columns}

def calculate_rms(window, eeg_columns):
    return {f'{col}_rms': np.sqrt(np.mean(window**2, axis=0)) for col in eeg_columns}

def calculate_skew_kurt(window, eeg_columns):
    mean = np.mean(window, axis=0)
    std = np.std(window, axis=0) + 1e-8  # Avoid division by zero
    z = (window - mean) / std
    skew = np.mean(z**3, axis=0)
    kurt = np.mean(z**4, axis=0) - 3  # Fisher's kurtosis
    return {f'{col}_skew': skew_val for col, skew_val in zip(eeg_columns, skew)} | \
           {f'{col}_kurt': kurt_val for col, kurt_val in zip(eeg_columns, kurt)}

def calculate_zero_crossing(window, eeg_columns):
    sign_changes = np.diff(np.sign(window), axis=0)
    return {f'{col}_zcross': np.sum(sign_changes != 0, axis=0) for col in eeg_columns}

### Sliding Window Processing Function
def sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, feature_func, output_file):
    feature_list = []
    n_samples = eeg_data.shape[0]

    for start in range(0, n_samples - window_size + 1, step_size):
        end = start + window_size
        window = eeg_data[start:end]
        feature_list.append(feature_func(window, eeg_columns))

    # Convert to DataFrame and save
    feature_df = pd.DataFrame(feature_list)
    feature_df.to_csv(output_file, index=False)
    print(f"Feature extraction complete. Saved to {output_file}")

### Extract & Save Features Separately
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_mean, f"{output_dir}eeg_mean.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_variance, f"{output_dir}eeg_variance.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_std, f"{output_dir}eeg_std.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_min_max, f"{output_dir}eeg_min_max.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_ptp, f"{output_dir}eeg_ptp.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_energy, f"{output_dir}eeg_energy.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_rms, f"{output_dir}eeg_rms.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_skew_kurt, f"{output_dir}eeg_skew_kurt.csv")
sliding_window_feature_calc(eeg_data, eeg_columns, window_size, step_size, calculate_zero_crossing, f"{output_dir}eeg_zero_crossing.csv")

print("✅ All time-domain features extracted and saved separately.")


Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_mean.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_variance.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_std.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_min_max.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_ptp.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_energy.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_rms.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTSeeg_skew_kurt.csv
Feature extraction complete. Saved to /Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_O

In [3]:
import numpy as np
import pandas as pd

# Load dataset
data = pd.read_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/EEG Assets/chbmit_preprocessed_data.csv')
eeg_columns = [col for col in data.columns if col != 'Outcome']
eeg_data = data[eeg_columns].values

# Compute MAD
mad_values = np.mean(np.abs(eeg_data - np.mean(eeg_data, axis=0)), axis=0)

# Save
mad_df = pd.DataFrame({f"{col}_mad": mad_values for col in eeg_columns})
mad_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/mad_features.csv' , index=False)

print("✅ MAD features saved!")


✅ MAD features saved!


In [4]:
def hjorth_params(signal):
    """Compute Hjorth Activity, Mobility, Complexity."""
    activity = np.var(signal, axis=0)  # Power of signal
    diff1 = np.diff(signal, axis=0)
    mobility = np.sqrt(np.var(diff1, axis=0) / activity)
    
    diff2 = np.diff(diff1, axis=0)
    complexity = np.sqrt(np.var(diff2, axis=0) / np.var(diff1, axis=0)) / mobility
    
    return activity, mobility, complexity

# Compute Hjorth parameters
activity, mobility, complexity = hjorth_params(eeg_data)

# Save separately
activity_df = pd.DataFrame({f"{col}_hjorth_activity": activity for col in eeg_columns})
mobility_df = pd.DataFrame({f"{col}_hjorth_mobility": mobility for col in eeg_columns})
complexity_df = pd.DataFrame({f"{col}_hjorth_complexity": complexity for col in eeg_columns})

activity_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/hjorth_activity.csv', index=False)
mobility_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/hjorth_mobility.csv', index=False)
complexity_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/hjorth_complexity.csv', index=False)

print("✅ Hjorth features saved!")


✅ Hjorth features saved!


In [5]:
line_length = np.sum(np.abs(np.diff(eeg_data, axis=0)), axis=0)

# Save
line_length_df = pd.DataFrame({f"{col}_line_length": line_length for col in eeg_columns})
line_length_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/line_length.csv', index=False)

print("✅ Line Length features saved!")


✅ Line Length features saved!


In [6]:
def shannon_entropy(signal):
    """Compute Shannon entropy for each channel."""
    prob = signal / np.sum(signal, axis=0)  # Normalize
    entropy = -np.sum(prob * np.log2(prob + 1e-10), axis=0)  # Avoid log(0)
    return entropy

# Compute entropy
entropy_values = shannon_entropy(np.abs(eeg_data))

# Save
entropy_df = pd.DataFrame({f"{col}_shannon_entropy": entropy_values for col in eeg_columns})
entropy_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS/shannon_entropy.csv', index=False)

print("✅ Shannon Entropy saved!")


✅ Shannon Entropy saved!


In [9]:
import os
import numpy as np
import pandas as pd

# Define save path
save_dir = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME_FEATURES_OUTPUTS"
save_path = os.path.join(save_dir, "permutation_entropy.csv")

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Compute entropy for each EEG channel
perm_entropy_values = np.array([permutation_entropy(eeg_data[:, i]) for i in range(eeg_data.shape[1])])

# Save
perm_entropy_df = pd.DataFrame({f"{col}_perm_entropy": perm_entropy_values for col in eeg_columns})
perm_entropy_df.to_csv(save_path, index=False)

print("✅ Permutation Entropy saved successfully!")


✅ Permutation Entropy saved successfully!
